# Multiprocessamento

O multiprocessmaneto permite correr código em paralelo, o que permite acelarar muito algumas tarefas no Python.

Exsitem duas formas para correr código em paralelo, com Threads e com Processos.

As Threads permitem correr código em paralelo, mas residem dentro do mesmo processo, que no caso do compilador Python significa que só corre num núcleo do processador (noutras linguagens tal não é verdade podendo várias Threads correr em diferentes núcleos do processador).

Já a utilização de processos em Python permite correr código em diferentes núcleos do processador.

In [ ]:
#Primeiro sem threads
def soma(worker):
  soma = 0
  for i in range(worker[0],worker[1]):
    soma = soma + i
  print(soma)

print("Sem threads")
soma([0,100])
soma([100,200])
soma([0,200])

Sem threads
4950
14950
19900


In [ ]:
import os
#Repetir o exercício anterior com o número do processo
def soma(worker):
  print("pid: %s" % os.getpid())
  soma = 0
  for i in range(worker[0],worker[1]):
    soma = soma + i
  print(soma)

print("Sem threads")
soma([0,100])
soma([100,200])
soma([0,200])

Sem threads
pid: 62
4950
pid: 62
14950
pid: 62
19900


In [ ]:
import os, threading, queue

print("Com threads")

def threadTest(worker):
  soma = 0
  for i in range (worker[0],worker[1]):
    soma = soma + i
  print(" " + str(soma)) 

t1 = threading.Thread(target = threadTest, args=([0,100],))
t1.start()
t2 = threading.Thread(target = threadTest, args=([100,200],))
t2.start()
t3 = threading.Thread(target = threadTest, args=([0,200],))
t3.start()

t1.join()
t2.join()
t3.join()

Com threads
 4950
 14950
 19900


In [ ]:
import os, time
#Repetir o exercício anterior com o número do processo
print("Com threads")

def threadTest(worker):
  print("pid: %s" % os.getpid())
  soma = 0
  for i in range (worker[0],worker[1]):
    soma = soma + i
    time.sleep(0.05)
  print(soma) 

t1 = threading.Thread(target = threadTest, args=([0,100],))
t1.start()
t2 = threading.Thread(target = threadTest, args=([100,200],))
t2.start()
t3 = threading.Thread(target = threadTest, args=([0,200],))
t3.start()

t1.join()
t2.join()
t3.join()

Com threads
pid: 62
pid: 62pid: 62

495014950

19900


In [ ]:
print("Com threads")
print_lock = threading.Lock()

def threadTest(worker):
  with print_lock:
    soma = 0
    for i in range (worker[0],worker[1]):
      soma = soma + i
    print(soma) 

def threader():
  while True:
    threadTest(q.get())
    q.task_done()

q = queue.Queue()
for x in range(5):
  t = threading.Thread(target = threader)
  t.daemon = True
  t.start()

q.put([0,100])
q.put([100,200])
q.put([0,200])

q.join()

Com threads
4950
14950
19900


In [ ]:
print("Com threads")
print_lock = threading.Lock()

def threadTest(worker):
  with print_lock:
    soma = 0
    for i in range (worker[0],worker[1]):
      soma = soma + i
  return soma

def threader():
  while True:
    q_out.put(threadTest(q_in.get()))
    q_in.task_done()
  return q

q_in = queue.Queue()
q_out = queue.Queue()

for x in range(1):
  t = threading.Thread(target = threader)
  t.daemon = True
  t.start()

q_in.put([0,100])
q_in.put([100,200])
q_in.join()

#Juntar os dois valores das threads
resultado = 0
while not q_out.empty():
  resultado = resultado + q_out.get()

print(resultado)

q_in.put([0,200])
q_in.join()
print(q_out.get())




Com threads
19900
19900


In [ ]:
#Com processos
import multiprocessing
def spawn():
  print("pid: %s" % os.getpid())
  print('test!')

for i in range(5):
  p = multiprocessing.Process(target=spawn)
  p.start()

pid: 288
pid: 285
test!
test!
pid: 291
test!
pid: 299
pid: 292
test!
test!


In [ ]:
#Com processos
import multiprocessing

def soma(worker):
  soma = 0
  for i in range (worker[0],worker[1]):
    soma = soma + i
  print(soma)

p1 = multiprocessing.Process(target=soma, args=([0,100],))
p1.start()
p2 = multiprocessing.Process(target=soma, args=([100,200],))
p2.start()

4950
14950


In [ ]:
#Com processos
import multiprocessing, os

def soma(worker):
  print("pid: %s" % os.getpid())
  soma = 0
  for i in range (worker[0],worker[1]):
    soma = soma + i
  print(soma)

for x in range (0,2000,100):
  p = multiprocessing.Process(target=soma, args=([x,100+x],))
  p.start()


#for x in range (0,1000,100):
#  p = multiprocessing.Process(target=soma, args=([x,100+x],))
#  p.start()
#  p.join()

pid: 514
4950
pid: 517
14950
pid: 520
24950
pid: 525
pid: 532
34950
44950
pid: 537
pid: 540
54950
pid: 543
pid: 548
64950
74950
pid: 558
104950
pid: 553
84950
94950
pid: 563
114950
pid: 570
pid: 578
144950
pid: 575
124950
134950
pid: 587
154950
pid: 593
pid: 592
164950
174950
pid: 600
pid: 607
194950
184950


In [ ]:
#Com processos
import multiprocessing

def soma(worker):
  soma = 0
  for i in range (worker[0],worker[1]):
    soma = soma + i
  print(soma)

for x in range (0,1000,100):
  p = multiprocessing.Process(target=soma, args=([x,100+x],))
  p.start()
  p.join()

4950
14950
24950
34950
44950
54950
64950
74950
84950
94950


In [ ]:
#Para devolver valores utilizar a instrução Pool
import multiprocessing
from functools import reduce
from operator import add

def soma(lim_inf):
  print("pid: %s" % os.getpid())
  soma = 0
  for i in range (lim_inf,lim_inf+100):
    soma = soma + i
  return soma

with multiprocessing.Pool(2) as p:
  results = p.map(soma, range(0,200,100))
  print(results)
  print(reduce(add, results)) #aqui também podem fazer um ciclo for, mas lembrei-me de utilizar esta função
  p.close()
  p.join()


pid: 1496
pid: 1497
[4950, 14950]
19900


In [ ]:
#Alterar a funcao anterior para 10 processos com um sleep de 5 segundos e medir o tempo

Para saber mais sobre as diferenças entre threads e processos consultem https://timber.io/blog/multiprocessing-vs-multithreading-in-python-what-you-need-to-know/

In [ ]:
#Para devolver valores utilizar a instrução Pool
import multiprocessing
from functools import reduce
from operator import add
import time

def soma(lim_inf):
  print("pid: %s" % os.getpid())
  soma = 0
  for i in range (lim_inf,lim_inf+100):
    soma = soma + i
    time.sleep(0.01)
  return soma

starttime =  time.time()

with multiprocessing.Pool(10) as p:
  results = p.map(soma, range(0,1000,100))
  print(results)
  print(reduce(add, results)) #aqui também podem fazer um ciclo for, mas lembrei-me de utilizar esta função
  p.close()
  p.join()

endtime =  time.time()

print ("Tempo de execução: %s" % (endtime-starttime))

starttime =  time.time()

for i in range(0,1000,100):
  soma(i)

endtime =  time.time()

print ("Tempo de execução: %s" % (endtime-starttime))



pid: 939
pid: 940
pid: 943
pid: 945
pid: 947
pid: 944
pid: 946
pid: 942
pid: 938
pid: 941
[4950, 14950, 24950, 34950, 44950, 54950, 64950, 74950, 84950, 94950]
499500
Tempo de execução: 1.2123100757598877
pid: 62
pid: 62
pid: 62
pid: 62
pid: 62
pid: 62
pid: 62
pid: 62
pid: 62
pid: 62
Tempo de execução: 10.153066635131836
